In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# to read .csv files from another directory
location = "detect-pneumonia-spring-2024/" # "/<path>"

df = pd.read_csv(location + "labels_train.csv")
df

/home/lazaros/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2024-04-07 17:41:46.252012: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 17:41:46.299917: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 17:41:46.384335: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 17:41:47.3722

,file_name,class_id
0,img_4358977458434011046.jpg,1
1,img_5224016757187192130.jpg,2
2,img_3065202206106254707.jpg,2
3,img_6304894865561547174.jpg,1
4,img_3371338542810939877.jpg,2
...,...,...
4667,img_6661311872293090412.jpg,2
4668,img_3844568579349757418.jpg,1
4669,img_9145812369383814369.jpg,1
4670,img_1311393330250392648.jpg,1


In [2]:
df['class_id'].value_counts()

class_id
1    2238
0    1227
2    1207
Name: count, dtype: int64

In [3]:
# Define the target number of images per class
target_images_per_class = df['class_id'].value_counts().max()
target_images_per_class

2238

In [4]:
# Define augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=10,  # rotate images by up to 10 degrees
    width_shift_range=0.1,  # shift images horizontally by up to 10% of the width
    height_shift_range=0.1,  # shift images vertically by up to 10% of the height
    shear_range=0.2,  # shear intensity (shear angle in radians)
    zoom_range=0.1,  # zoom range [1-zoom_range, 1+zoom_range]
    horizontal_flip=True,  # horizontally flip images
    vertical_flip=False,  # don't vertically flip images (not applicable for X-ray images)
    fill_mode='nearest'  # fill mode for points outside the input boundaries
)

# Specify the output folder for the augmented images
output_folder = location + "train_images/train_augmented_images_resized/"
input_folder = location + "train_images/train_images_resized/"


# Iterate through each class
for class_id in df['class_id'].unique():
    # Select images belonging to the current class
    class_images = df[df['class_id'] == class_id]['file_name'].tolist()
    
    # Check if the current class needs additional images
    if len(class_images) < target_images_per_class:
        # Calculate the number of additional images to generate
        num_additional_images = target_images_per_class - len(class_images)
        
        # Create a subfolder for the current class
        class_output_folder = os.path.join(output_folder, f"class_{class_id}")
        if not os.path.exists(class_output_folder):
            os.makedirs(class_output_folder)
        
        # Generate additional augmented images for this class
        for image_name in class_images:
            image_path = os.path.join(input_folder, image_name)  # Adjust path as needed
            img = cv2.imread(image_path)
            img = img.reshape((1,) + img.shape)  # Reshape to 4D array (batch_size, height, width, channels)
            i = 0
            for batch in datagen.flow(img, batch_size=1, save_to_dir=class_output_folder, save_prefix='aug', save_format='jpg'):
                i += 1
                if i >= num_additional_images:
                    break  # Exit the loop after generating the specified number of additional images
        
print("Additional augmented images generated successfully!")


KeyboardInterrupt: 